In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# Load YOLOv3-tiny
yolo = cv2.dnn.readNet("yolov3.weights","yolov3.cfg.txt")

In [3]:
# Load COCO classes
with open("coco.names.txt", 'r') as f:
    classes = f.read().splitlines()

In [4]:
# Get output layer names
layer_names = yolo.getLayerNames()

# get output layer use final detectiion
out_layers = yolo.getUnconnectedOutLayers()
if isinstance(out_layers[0], (list, np.ndarray)):
    output_layers = [layer_names[i[0] - 1] for i in out_layers]
else:
    output_layers = [layer_names[i - 1] for i in out_layers]


In [5]:
# Colors for each class
colors = np.random.uniform(0, 255, size=(len(classes), 3))


In [7]:
# Start Video Capture 
cap = cv2.VideoCapture("demo2.MP4")    # "0"--->for webcamara

frame_id = 0  # for skipping frames

while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame,(600,600))
    if not ret:
        break

    frame_id += 1

    # Skip every odd-numbered frame to processing speed double and smoothness detection.
    if frame_id % 2 != 0:
        continue

    # Stores frame to height, width, and channel information for scaling detections.
    height, width, channels = frame.shape

    # Create blob from input frame (reduced size for faster processing)
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (320, 320), swapRB=True, crop=False)         #swapRB--->True changes BGR (OpenCV default) to RGB (YOLO expects RGB)

                                                                                            # crop--->False keeps original aspect ratio.
    # input for detection
    yolo.setInput(blob)

    # Forward pass
    outs = yolo.forward(output_layers)

     # Data for detected objects
    class_ids = []
    confidences = []
    boxes = []

    # Process outputs
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # confidence threshold
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Non-max use to remove overlapping boxes
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Draw boxes
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = colors[class_ids[i]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, f"{label} {int(confidence*100)}%", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Show frame
    cv2.imshow("YOLOv3-tiny Real-time Detection", frame)

    # Exit on P
    if cv2.waitKey(25) & 0xff == ord("p"):
        break

cap.release()
cv2.destroyAllWindows()
